In [8]:
# !pip install tensorflow==1.15.0
# !pip install bert-serving-server
# !pip install bert-serving-client 
!bert-serving-start -model_dir ./chinese_L-12_H-768_A-12/ -num_worker=2

^C


In [9]:
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf
tf.__version__

'1.15.0'

In [10]:
from bert_serving.client import BertClient

In [13]:
job=pd.read_excel("job_info_split.xlsx")
job=job.drop("Unnamed: 0",axis=1)
course=pd.read_excel("course_info_split.xlsx")
course=course.drop("Unnamed: 0",axis=1)

In [14]:
import random
idx=random.sample(range(len(job)), 100000 )
job2=pd.DataFrame(job.values[idx], columns=job.columns)
print("job隨機採計數：", len(job2))
#job2.head()

print("===確認隨機狀況==============================")
print("===不同年份資料數：")
print("2015:", len(job2[job2["yyyymm"]==201501]))
print("2016:", len(job2[job2["yyyymm"]==201601]))
print("2017:", len(job2[job2["yyyymm"]==201701]))
print("2018:", len(job2[job2["yyyymm"]==201801]))
print("===不同職類資料數：(職類/隨機取樣數/取樣數與元資料數比值)")
for i in list(set(job["jobcat"])):
    print(i,":",len(job2[job2["jobcat"]==i]), "||", round(len(job2[job2["jobcat"]==i])/len(job[job["jobcat"]==i]),3))
    
job2=job2[["index","斷詞_feature"]]
course2=course[["index","斷詞_feature"]]

feature_df=pd.concat([job2,course2], axis=0, ignore_index=True)
#feature_df.to_excel("feature_index_split.xlsx")

job隨機採計數： 100000
===確認隨機狀況==============================
===不同年份資料數：
2015: 24878
2016: 26113
2017: 26835
2018: 22174
===不同職類資料數：(職類/隨機取樣數/取樣數與元資料數比值)
其他類人員 : 245 || 0.285
餐飲類人員 : 2341 || 0.31
行政∕總務類人員 : 5991 || 0.301
運輸物流類人員 : 899 || 0.277
製圖∕測量類人員 : 3072 || 0.314
MIS∕網管類人員 : 2123 || 0.301
經營∕幕僚類人員 : 1601 || 0.297
保全類人員 : 334 || 0.29
門市營業類人員 : 4062 || 0.301
人力資源類人員 : 1574 || 0.309
軍警消防類人員 : 142 || 0.283
製程規劃類人員 : 1485 || 0.304
工程研發類人員 : 5062 || 0.303
法務∕智財類人員 : 664 || 0.315
專案∕產品管理類人員 : 1189 || 0.295
維修∕技術服務類人員 : 2114 || 0.315
客戶服務類人員 : 1850 || 0.297
學術研究類人員 : 234 || 0.319
財務∕會計∕稅務類 : 4615 || 0.304
環境安全衛生類人員 : 1542 || 0.304
生技∕醫療研發類人員 : 394 || 0.299
行銷類人員 : 5726 || 0.306
營建施作類人員 : 1411 || 0.306
生產管理類人員 : 1415 || 0.31
旅遊休閒類人員 : 1850 || 0.315
化工材料研發類人員 : 380 || 0.298
農林漁牧相關類人員 : 45 || 0.285
產品企劃類人員 : 839 || 0.307
營建規劃類人員 : 1514 || 0.313
品保∕品管類人員 : 2482 || 0.303
貿易類人員 : 1111 || 0.307
記者及採訪類人員 : 136 || 0.311
醫療保健服務人員 : 491 || 0.304
醫療專業人員 : 4456 || 0.302
操作∕技術類人員 : 2062 || 0.307
金融專業相關類人員 

In [15]:
ff=list(feature_df["斷詞_feature"])

### bert_by_keras

In [ ]:
#MAX_SEQ_LEN = 100
###################
# 如執行此程式碼時，出現numpy的錯誤，需要更新numpy
# pip install -U numpy
###################

import os
from keras_bert import load_trained_model_from_checkpoint

BERT_PRETRAINED_DIR = './chinese_L-12_H-768_A-12' # need to rewrite if you put model in other directory
config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file)#, seq_len=MAX_SEQ_LEN)

model.summary()

import codecs
import numpy as np
from keras_bert import Tokenizer

# build tokenizer
dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)     
tokenizer = Tokenizer(token_dict)

bert_vec, dev_vec = [], []
for i in final_tweet:
#    tokens = tokenizer.tokenize(i)
#    print('Tokens:', tokens)
#    print('==============================================================')
    indices, segments = tokenizer.encode(first=i, max_len=MAX_SEQ_LEN)

# extract vectors
    predicts = model.predict([np.array([indices]), np.array([segments])])[0]
    bert_vec.append(predicts.tolist())

### bert_by_testing_model

In [ ]:
from datetime import datetime

#bert_embedding=[]
for i in range(103277, len(ff)):
    #if i %100==0:
    print(i, "||", datetime.now().strftime("%H:%M:%S"))
    bc = BertClient()
    x=bc.encode(ff[i].split())
    bert_embedding.append(sum(x)/x.shape[0])

103277 || 13:25:23
103278 || 13:25:31
103279 || 13:25:38
103280 || 13:25:46
103281 || 13:25:53
103282 || 13:26:01
103283 || 13:26:03
103284 || 13:26:07
103285 || 13:26:12
103286 || 13:26:15
103287 || 13:26:18
103288 || 13:26:18
103289 || 13:26:19
103290 || 13:26:22
103291 || 13:26:25
103292 || 13:26:28
103293 || 13:26:31
103294 || 13:26:35
103295 || 13:26:38
103296 || 13:26:41
103297 || 13:26:45
103298 || 13:26:48
103299 || 13:26:54
103300 || 13:26:59
103301 || 13:27:05
103302 || 13:27:11
103303 || 13:27:17
103304 || 13:27:23
103305 || 13:27:25
103306 || 13:27:29
103307 || 13:27:30
103308 || 13:27:35
103309 || 13:27:40
103310 || 13:27:46
103311 || 13:27:52
103312 || 13:27:58
103313 || 13:28:04
103314 || 13:28:14
103315 || 13:28:26
103316 || 13:28:38
103317 || 13:28:50
103318 || 13:29:03
103319 || 13:29:15
103320 || 13:29:16
103321 || 13:29:17
103322 || 13:29:18
103323 || 13:29:19
103324 || 13:29:20
103325 || 13:29:24
103326 || 13:29:28
103327 || 13:29:32
103328 || 13:29:35
103329 || 13

In [50]:
len(bert_embedding)

103276

In [ ]:
feature_df["bert_embedding"]=bert_embedding
feature_df.head()

In [ ]:
df= feature_df[["index", "bert_embedding"]]
#del feature_df
df.head()

In [ ]:
df.to_csv("feature_bertembedding_table.csv", encoding="utf8")

In [ ]:
cat=[]
for i in df.index:
    if "j" in i:
        cat.append("j")
    if "c" in i:
        cat.append("c")
df["cate"]=cat

feature_df_j=df[df["cate"]=="j"]
feature_df_j=feature_df_j.reset_index(drop=False)
feature_df_c2=df[df["cate"]=="c"]
feature_df_c2=feature_df_j.reset_index(drop=False)

#### 梭哈版

In [ ]:
# c_index=list(feature_df_c2["index"])
# j_index=list(feature_df_j["index"])
# print("course資料數：", len(c_index))
# print("job資料數：", len(j_index))

# print("=========================================")
# print("total training:", len(feature_df_c2))

# final_ar=np.zeros((len(feature_df_c2), len(feature_df_j)))
# for i in range(len(feature_df_c2)):
#     print("==========",i, "||", datetime.now().strftime("%H:%M:%S"))
#     for j in range(len(feature_df_j)):
#         cc=np.array(list(feature_df_c2.iloc[i,1:-1]))
#         cs=np.sum((cc+1)**2)
#         jj=np.array(list(feature_df_j.iloc[j,1:-1]))
#         js=np.sum((jj+1)**2)
#         d=round( np.sum(cc*jj)/np.sqrt(cs)*np.sqrt(js), 5)
#         print(p)
#         final_ar[i][j]=d

#### 目標課程版

In [6]:
cctarget="c38,c39,c40,c199,c200,c201,c8570,c8571,c8572,c8573,c8574,c8575,c8515,c8516,c8517,c8567,c8568,c8569,c245,c246,c247,c248,c249,c250"
cctarget=cctarget.split(",")
feature_df_c=pd.DataFrame()
for ct in cctarget:
    fd_c=feature_df_c2[feature_df_c2.index==ct]
    feature_df_c=pd.concat([feature_df_c, fd_c], axis=0)
feature_df_c=feature_df_c.reset_index(drop=False)

In [9]:
feature_df_c.head()

,index,excel,word,powerpoint,outlook,打字,autocad,windows,50,20,...,信息,儀控,淨土,研討論文,紗線,公開,微感測器,視光,製作人,cate
0,c38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
1,c39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
2,c40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
3,c199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c
4,c200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,c


In [10]:
feature_df_j.head()

,index,excel,word,powerpoint,outlook,打字,autocad,windows,50,20,...,信息,儀控,淨土,研討論文,紗線,公開,微感測器,視光,製作人,cate
0,j318812,0.123584,0.125578,0.149652,0.0,0.0,0.000000,0.234686,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
1,j176997,0.701427,0.712741,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
2,j306726,0.000000,0.000000,0.000000,0.0,0.0,0.577586,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
3,j31621,0.136484,0.138686,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j
4,j289343,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,j


In [11]:
c_index=list(feature_df_c["index"])
j_index=list(feature_df_j["index"])
print("course資料數：", len(c_index))
print("job資料數：", len(j_index))

print("=========================================")
print("total training:", len(feature_df_c))

final_ar=np.zeros((len(feature_df_c), len(feature_df_j)))
for i in range(len(feature_df_c)):
    print("============",i, "||", datetime.now().strftime("%H:%M:%S"))
    for j in range(len(feature_df_j)):
        cc=feature_df_c.loc[i,"bert_embedding"]
        cs=np.sum((cc+1)**2)
        jj=feature_df_j.loc[j,"bert_embedding"]
        js=np.sum((jj+1)**2)
        d=round(np.sum(cc*jj)/(np.sqrt(cs)*np.sqrt(js)),5)
        print(d)
        final_ar[i][j]=d

course資料數： 24
job資料數： 100000
total training: 24
0 || 21:41:09
1 || 21:51:45
2 || 22:17:55
3 || 22:28:29
4 || 22:39:10
5 || 22:49:53
6 || 23:04:07
7 || 23:18:51
8 || 23:32:41
9 || 23:43:40
10 || 23:54:34
11 || 00:05:26
12 || 00:16:10
13 || 00:27:07


KeyboardInterrupt: 

In [12]:
final_df=pd.DataFrame(final_ar.T, columns=c_index, index=j_index)

In [13]:
final_df.head()

,c38,c39,c40,c199,c200,c201,c8570,c8571,c8572,c8573,...,c8517,c8567,c8568,c8569,c245,c246,c247,c248,c249,c250
j318812,5015.83956,5015.88517,5015.86780,5015.89416,5015.96505,5016.05441,5016.23308,5016.28003,5016.26417,5016.29671,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
j176997,5012.83392,5012.87954,5012.86216,5012.88853,5012.95944,5013.04882,5013.22752,5013.27448,5013.25862,5013.29116,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
j306726,5012.77317,5012.81879,5012.80141,5012.82778,5012.89869,5012.98806,5013.16676,5013.21373,5013.19787,5013.23041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
j31621,5015.86932,5015.91493,5015.89756,5015.92392,5015.99481,5016.08417,5016.30783,5016.35647,5016.35375,5016.38637,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
j289343,5013.76906,5013.81468,5013.79730,5013.82367,5013.89457,5013.98394,5014.16263,5014.20959,5014.19373,5014.22627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
final_df.to_csv("bert_similarity_table.csv",encoding="utf8")